# Introduction

In this tutorial, we will go through an example to update a preexisting model. This might be useful when you come across additional data that you would want to consider, without having to train a model from scratch.

The main abstraction that Lightwood offers for this is the `BaseMixer.partial_fit()` method. To call it, you need to pass new training data and a held-out dev subset for internal mixer usage (e.g. early stopping). If you are using an aggregate ensemble, it's likely you will want to do this for every single mixer. The convienient `PredictorInterface.adjust()` does this automatically for you.


# Initial model training

First, let's train a Lightwood predictor for the `concrete strength` dataset:

In [1]:
from lightwood.api.high_level import ProblemDefinition, json_ai_from_problem, predictor_from_json_ai
import pandas as pd

INFO:lightwood-2387:No torchvision detected, image helpers not supported.
INFO:lightwood-2387:No torchvision/pillow detected, image encoder not supported
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/gluonts/model/deepar/__init__.py:18: FutureWarning: The module gluonts.model.deepar has been moved to gluonts.mx.model.deepar. In GluonTS v0.12 it will be no longer possible to use the old path. Try to use 'from gluonts.mx import DeepAREstimator'.
  warnings.warn(


In [2]:
# Load data
df = pd.read_csv('https://raw.githubusercontent.com/mindsdb/lightwood/staging/tests/data/concrete_strength.csv')

df = df.sample(frac=1, random_state=1)
train_df = df[:int(0.1*len(df))]
update_df = df[int(0.1*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

print(f'Train dataframe shape: {train_df.shape}')
print(f'Update dataframe shape: {update_df.shape}')
print(f'Test dataframe shape: {test_df.shape}')

Train dataframe shape: (103, 10)
Update dataframe shape: (721, 10)
Test dataframe shape: (206, 10)


Note that we have three different data splits.

We will use the `training` split for the initial model training. As you can see, it's only a 20% of the total data we have. The `update` split will be used as training data to adjust/update our model. Finally, the held out `test` set will give us a rough idea of the impact our updating procedure has on the model's predictive capabilities.

In [3]:
# Define predictive task and predictor
target = 'concrete_strength'
pdef = ProblemDefinition.from_dict({'target': target, 'time_aim': 200})
jai = json_ai_from_problem(df, pdef)

# We will keep the architecture simple: a single neural mixer, and a `BestOf` ensemble:
jai.model = {
    "module": "BestOf",
    "args": {
        "args": "$pred_args",
        "accuracy_functions": "$accuracy_functions",
        "submodels": [{
            "module": "Neural",
            "args": {
                "fit_on_dev": False,
                "stop_after": "$problem_definition.seconds_per_mixer",
                "search_hyperparameters": False,
            }
        }]
    }
}

# Build and train the predictor
predictor = predictor_from_json_ai(jai)
predictor.learn(train_df)

INFO:type_infer-2387:Analyzing a sample of 979
INFO:type_infer-2387:from a total population of 1030, this is equivalent to 95.0% of your data.
INFO:type_infer-2387:Infering type for: id
INFO:type_infer-2387:Column id has data type integer
INFO:type_infer-2387:Infering type for: cement
INFO:type_infer-2387:Column cement has data type float
INFO:type_infer-2387:Infering type for: slag
INFO:type_infer-2387:Column slag has data type float
INFO:type_infer-2387:Infering type for: flyAsh
INFO:type_infer-2387:Column flyAsh has data type float
INFO:type_infer-2387:Infering type for: water
INFO:type_infer-2387:Column water has data type float
INFO:type_infer-2387:Infering type for: superPlasticizer
INFO:type_infer-2387:Column superPlasticizer has data type float
INFO:type_infer-2387:Infering type for: coarseAggregate
INFO:type_infer-2387:Column coarseAggregate has data type float
INFO:type_infer-2387:Infering type for: fineAggregate
INFO:type_infer-2387:Column fineAggregate has data type float
I

In [4]:
# Train and get predictions for the held out test set
predictions = predictor.predict(test_df)
predictions

INFO:dataprep_ml-2387:[Predict phase 1/4] - Data preprocessing
INFO:dataprep_ml-2387:Cleaning the data
DEBUG:lightwood-2387: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2387:[Predict phase 2/4] - Feature generation
INFO:dataprep_ml-2387:Featurizing the data
DEBUG:lightwood-2387: `featurize` runtime: 0.0 seconds
INFO:dataprep_ml-2387:[Predict phase 3/4] - Calling ensemble
INFO:dataprep_ml-2387:[Predict phase 4/4] - Analyzing output
INFO:lightwood-2387:The block ICP is now running its explain() method
INFO:lightwood-2387:The block ConfStats is now running its explain() method
INFO:lightwood-2387:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2387:The block AccStats is now running its explain() method
INFO:lightwood-2387:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2387:The block PermutationFeatureImportance is now running its explain() method
INFO:l

,original_index,prediction,confidence,lower,upper
0,0,56.244863,0.9991,20.917343,91.572383
1,1,35.427652,0.9991,0.100132,70.755172
2,2,13.515624,0.9991,0.000000,48.843145
3,3,4.662318,0.9991,0.000000,39.989838
4,4,40.978188,0.9991,5.650668,76.305708
...,...,...,...,...,...
201,201,47.370314,0.9991,12.042793,82.697834
202,202,38.840124,0.9991,3.512604,74.167645
203,203,33.546998,0.9991,0.000000,68.874518
204,204,22.786796,0.9991,0.000000,58.114317


## Updating the predictor

For this, we have two options:

### `BaseMixer.partial_fit()`

Updates a single mixer. You need to pass the new data wrapped in `EncodedDs` objects.

**Arguments:** 
* `train_data: EncodedDs`
* `dev_data: EncodedDs`
* `adjust_args: Optional[dict]` - This will contain any arguments needed by the mixer to adjust new data.

If the mixer does not need a `dev_data` partition, pass a dummy:

```
dev_data = EncodedDs(predictor.encoders, pd.DataFrame(), predictor.target)
```

### `PredictorInterface.adjust()`

Updates all mixers inside the predictor by calling their respective `partial_fit()` methods. Any `adjust_args` will be transparently passed as well.

**Arguments:**

* `new_data: pd.DataFrame`
* `old_data: Optional[pd.DataFrame]`
* `adjust_args: Optional[dict]`

Let's `adjust` our predictor:

In [5]:
predictor.adjust(update_df, train_df)  # data to adjust and original data

INFO:dataprep_ml-2387:Cleaning the data
DEBUG:lightwood-2387: `preprocess` runtime: 0.03 seconds
INFO:dataprep_ml-2387:Cleaning the data
DEBUG:lightwood-2387: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2387:Updating the mixers
torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
INFO:lightwood-2387:Loss @ epoch 1: 0.02191816394527753
DEBUG:lightwood-2387: `adjust` runtime: 4.16 seconds


In [6]:
new_predictions = predictor.predict(test_df)
new_predictions

INFO:dataprep_ml-2387:[Predict phase 1/4] - Data preprocessing
INFO:dataprep_ml-2387:Cleaning the data
DEBUG:lightwood-2387: `preprocess` runtime: 0.01 seconds
INFO:dataprep_ml-2387:[Predict phase 2/4] - Feature generation
INFO:dataprep_ml-2387:Featurizing the data
DEBUG:lightwood-2387: `featurize` runtime: 0.0 seconds
INFO:dataprep_ml-2387:[Predict phase 3/4] - Calling ensemble
INFO:dataprep_ml-2387:[Predict phase 4/4] - Analyzing output
INFO:lightwood-2387:The block ICP is now running its explain() method
INFO:lightwood-2387:The block ConfStats is now running its explain() method
INFO:lightwood-2387:ConfStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2387:The block AccStats is now running its explain() method
INFO:lightwood-2387:AccStats.explain() has not been implemented, no modifications will be done to the data insights.
INFO:lightwood-2387:The block PermutationFeatureImportance is now running its explain() method
INFO:l

,original_index,prediction,confidence,lower,upper
0,0,56.749161,0.9991,21.421641,92.076681
1,1,36.211965,0.9991,0.884445,71.539486
2,2,14.090568,0.9991,0.000000,49.418088
3,3,5.227239,0.9991,0.000000,40.554759
4,4,41.558701,0.9991,6.231181,76.886221
...,...,...,...,...,...
201,201,47.878132,0.9991,12.550612,83.205652
202,202,39.038255,0.9991,3.710735,74.365775
203,203,34.124498,0.9991,0.000000,69.452018
204,204,23.070268,0.9991,0.000000,58.397789


Nice! Our predictor was updated, and new predictions are looking good. Let's compare the old and new accuracies to complete the experiment:

In [7]:
from sklearn.metrics import r2_score
import numpy as np

old_acc = r2_score(test_df['concrete_strength'], predictions['prediction'])
new_acc = r2_score(test_df['concrete_strength'], new_predictions['prediction'])

print(f'Old Accuracy: {round(old_acc, 3)}\nNew Accuracy: {round(new_acc, 3)}')

Old Accuracy: 0.784
New Accuracy: 0.782


## Conclusion

We have gone through a simple example of how Lightwood predictors can leverage newly acquired data to improve their predictions. The interface for doing so is fairly simple, requiring only some new data and a single call to update.

You can further customize the logic for updating your mixers by modifying the `partial_fit()` methods in them.